In [1]:
from gurobipy import *
import pandas as pd

In [2]:
# df1_1 = pd.read_excel("OR110-1_case01.xlsx", "Instance 1")
# df1_2 = pd.read_excel("OR110-1_case01.xlsx", "Instance 2")
df1_3 = pd.read_excel("OR110-1_case01.xlsx", "Instance 3")

# display(df1_3)

df1_3 = df1_3.rename(columns={'Process Type': 'type_P1',
                              'Unnamed: 2': 'type_P2',
                              'Unnamed: 3': 'type_P3', 
                              'Unnamed: 4': 'type_P4', 
                              'Unnamed: 5': 'type_P5',
                              'Processing Time': 'time_P1', 
                              'Unnamed: 8': 'time_P2',
                              'Unnamed: 9': 'time_P3',
                              'Unnamed: 10': 'time_P4',
                              'Unnamed: 11': 'time_P5',
                             })
df1_3 = df1_3.drop(0)

df1_3 = df1_3.fillna(0)

# display(df1_1)
# display(df1_2)
display(df1_3)

,Job ID,type_P1,type_P2,type_P3,type_P4,type_P5,Splitting Timing,time_P1,time_P2,time_P3,time_P4,time_P5,Due Time
1,1.0,Boiling,Baking,Smoking,0,0,2.0,2.0,1.0,1.5,0.0,0.0,12:30:00
2,2.0,Boiling,Baking,Boiling,0,0,2.0,1.6,1.4,2.0,0.0,0.0,12:30:00
3,3.0,Boiling,Baking,Boiling,Baking,Boiling,1.0,3.0,0.8,1.1,0.3,1.2,12:30:00
4,4.0,Smoking,Boiling,Baking,Smoking,Boiling,4.0,0.3,1.2,1.0,0.5,0.6,12:30:00
5,5.0,Boiling,Baking,Boiling,0,0,1.0,1.2,0.7,1.8,0.0,0.0,12:30:00
6,6.0,Boiling,0,0,0,0,0.0,3.3,0.0,0.0,0.0,0.0,12:30:00
7,7.0,Baking,Smoking,0,0,0,1.0,1.7,1.5,0.0,0.0,0.0,17:30:00
8,8.0,Baking,0,0,0,0,0.0,1.3,0.0,0.0,0.0,0.0,17:30:00
9,9.0,Boiling,Baking,Boiling,0,0,2.0,0.8,1.0,2.2,0.0,0.0,17:30:00
10,10.0,Baking,Smoking,Boiling,Baking,Smoking,3.0,0.6,1.3,0.7,0.2,1.1,17:30:00


In [3]:
n = len(df1_3["Job ID"])
J = range(n)
I = range(4)
# K = [(j, k) for j in J for k in J if j < k]
H = [(j, k) for j in J for k in J if j != k]

Pn = 5

P = []
for i in J:
    idx = 1
    tmpP = 0
    tmpStr = "time_P" + str(idx)
    while idx<=Pn and df1_3[tmpStr][i+1] != 0:
        tmpP += df1_3[tmpStr][i+1]
        idx += 1
        tmpStr = "time_P" + str(idx)
    P.append(tmpP)
    
display(P)

M1 = 0
for i in P:
    M1 += i

[4.5, 5.0, 6.4, 3.6, 3.7, 3.3, 3.2, 1.3, 4.0, 3.9]

In [4]:
import datetime

start = datetime.time(7, 30, 0)
start = datetime.datetime.combine(datetime.date.today(), start)

D = []

for i in df1_3["Due Time"]:
    a = datetime.datetime.combine(datetime.date.today(), i) - start
    D.append(int(a.seconds / 3600))
    #due.append(datetime.combine(date.today(), i) - datetime.combine(date.today(), start))

print(D)

[5, 5, 5, 5, 5, 5, 10, 10, 10, 10]


In [5]:
eg2 = Model("problem1")    # build a new model

x = eg2.addVars(I, J, lb = 0, vtype = GRB.BINARY, name = "x")
z = eg2.addVars(H, lb = 0, vtype = GRB.BINARY, name = "z")
t = eg2.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f = eg2.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-28


In [6]:
# setting the objective function 
eg2.setObjective(
    quicksum(t[j] for j in J)
    , GRB.MINIMIZE) 


# add constraints and name them

for j in J:
    eg2.addConstr((f[j] - D[j] <= M1 * t[j]), name = f"define t_{j+1}")
    
for j in J:
    eg2.addConstr(quicksum(x[i, j] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg2.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg2.addConstr(z[j, k] + z[k, j] + 1 >= x[i, j] + x[i, k], name = f"define z_{j+1}_{k+1}")
#         eg2.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg2.addConstr(f[j] + P[k] - f[k] <= M1 * (1 - z[j, k]), name = f"define f_{k+1}")
#     eg2.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j+1}")

for j in J:
    eg2.addConstr(f[j] >= P[j], name = f"f_{j+1} > P_{j+1}")
    
eg2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 480 rows, 150 columns and 1780 nonzeros
Model fingerprint: 0x890cd259
Variable types: 10 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 4.0000000
Presolve removed 191 rows and 1 columns
Presolve time: 0.00s
Presolved: 289 rows, 149 columns, 1048 nonzeros
Variable types: 10 continuous, 139 integer (139 binary)

Root relaxation: objective 1.000000e+00, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    2    4.00000    1.00000  75.0%     -    0s
H    0     0                     

In [7]:
print("Result:")

# objecitve value
print("z* =", eg2.objVal)
print("")

# t
for j in J:
    print(f[j].varName, '=', round(f[j].x, 2), ', Due time =', D[j], ',', t[j].varName, '=', t[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x[i, j].x)) < 7:
            print(x[i, j].x, "\t", end="")
        else:
            print(y[i, j].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="")
for j in J:
    print("\tJob" + str(j+1), end="")
print("")    # use for change line
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for k in J:
        # print values of each kind of product
        if j != k:
            if len(str(z[j, k].x)) < 7:
                print(z[j, k].x, "\t", end="")
            else:
                print(z[j, k].x, "", end="")
        else:
            print("\t", end="")
    print("")    # use for change line
print("")

Result:
z* = 2.0

f[0] = 7.8 , Due time = 5 , t[0] = 1.0
f[1] = 5.0 , Due time = 5 , t[1] = 0.0
f[2] = 15.4 , Due time = 5 , t[2] = 1.0
f[3] = 3.6 , Due time = 5 , t[3] = 0.0
f[4] = 5.0 , Due time = 5 , t[4] = 0.0
f[5] = 3.3 , Due time = 5 , t[5] = 0.0
f[6] = 8.2 , Due time = 10 , t[6] = 0.0
f[7] = 1.3 , Due time = 10 , t[7] = 0.0
f[8] = 9.0 , Due time = 10 , t[8] = 0.0
f[9] = 7.5 , Due time = 10 , t[9] = 0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1 	0.0 	0.0 	0.0 	1.0 	
Job2 	0.0 	1.0 	0.0 	0.0 	
Job3 	0.0 	1.0 	0.0 	0.0 	
Job4 	1.0 	0.0 	0.0 	0.0 	
Job5 	0.0 	0.0 	1.0 	0.0 	
Job6 	0.0 	0.0 	0.0 	1.0 	
Job7 	0.0 	0.0 	1.0 	0.0 	
Job8 	0.0 	0.0 	1.0 	0.0 	
Job9 	0.0 	1.0 	0.0 	0.0 	
Job10 	1.0 	0.0 	0.0 	0.0 	

z:
j \ k	Job1	Job2	Job3	Job4	Job5	Job6	Job7	Job8	Job9	Job10
Job1 		0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	
Job2 	0.0 		1.0 	0.0 	0.0 	0.0 	0.0 	0.0 	1.0 	0.0 	
Job3 	0.0 	0.0 		0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	
Job4 	0.0 	0.0 	1.0 		0.0 	0.0 	1.0 	0.0 	0.0 	1.0

In [8]:
eg3 = Model("problem1-2")    # build a new model

x = eg3.addVars(I, J, lb = 0, vtype = GRB.BINARY, name = "x")
z = eg3.addVars(J, J, lb = 0, vtype = GRB.BINARY, name = "z")
t = eg3.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f = eg3.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

w = eg3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

In [9]:
# setting the objective function 
eg3.setObjective(
    w
    , GRB.MINIMIZE) 


# add constraints and name them

for j in J:
    eg3.addConstr(w >= f[j], name="w")

eg3.addConstr(quicksum(t[j] for j in J) <= eg2.objVal, name = "limit tardy jobs")

for j in J:
    eg3.addConstr((f[j] - D[j] <= M1 * t[j]), name = f"define t_{j+1}")
    
for j in J:
    eg3.addConstr(quicksum(x[i, j] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg3.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg3.addConstr(z[j, k] + z[k, j] + 1 >= x[i, j] + x[i, k], name = f"define z_{j+1}_{k+1}")
#         eg3.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg3.addConstr(f[j] + P[k] - f[k] <= M1 * (1 - z[j, k]), name = f"define f_{k+1}")
#     eg3.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j+1}")

for j in J:
    eg3.addConstr(f[j] >= P[j], name = f"f_{j+1} > P_{j+1}")
    
eg3.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 491 rows, 161 columns and 1810 nonzeros
Model fingerprint: 0xd6f2f216
Variable types: 11 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 192 rows and 12 columns
Presolve time: 0.00s
Presolved: 299 rows, 149 columns, 1075 nonzeros
Variable types: 11 continuous, 138 integer (138 binary)

Root relaxation: objective 6.400000e+00, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.40000    0   14          -    6.40000      -     -    0s
H    0     0                      19.9000000    6.40000  67.8%     -    0s
H  

In [10]:
print("Result:")

# objecitve value
print("z* =", round(eg3.objVal, 2))
print("")

# t
for j in J:
    print(f[j].varName, '=', round(f[j].x, 2), ', Due time =', D[j], ',', t[j].varName, '=', t[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x[i, j].x)) < 7:
            print(x[i, j].x, "\t", end="")
        else:
            print(y[i, j].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="")
for j in J:
    print("\tJob" + str(j+1), end="")
print("")    # use for change line
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for k in J:
        # print values of each kind of product
        if j != k:
            if len(str(z[j, k].x)) < 7:
                print(z[j, k].x, "\t", end="")
            else:
                print(z[j, k].x, "", end="")
        else:
            print("\t", end="")
    print("")    # use for change line
print("")

Result:
z* = 10.1

f[0] = 4.5 , Due time = 5 , t[0] = -0.0
f[1] = 5.0 , Due time = 5 , t[1] = 0.0
f[2] = 10.1 , Due time = 5 , t[2] = 1.0
f[3] = 10.1 , Due time = 5 , t[3] = 1.0
f[4] = 3.7 , Due time = 5 , t[4] = -0.0
f[5] = 3.3 , Due time = 5 , t[5] = -0.0
f[6] = 6.5 , Due time = 10 , t[6] = -0.0
f[7] = 9.8 , Due time = 10 , t[7] = -0.0
f[8] = 8.5 , Due time = 10 , t[8] = -0.0
f[9] = 8.9 , Due time = 10 , t[9] = -0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1 	1.0 	-0.0 	-0.0 	0.0 	
Job2 	0.0 	-0.0 	1.0 	0.0 	
Job3 	-0.0 	1.0 	0.0 	-0.0 	
Job4 	0.0 	0.0 	0.0 	1.0 	
Job5 	0.0 	1.0 	-0.0 	0.0 	
Job6 	0.0 	0.0 	0.0 	1.0 	
Job7 	0.0 	0.0 	-0.0 	1.0 	
Job8 	1.0 	0.0 	0.0 	0.0 	
Job9 	1.0 	0.0 	-0.0 	-0.0 	
Job10 	-0.0 	-0.0 	1.0 	0.0 	

z:
j \ k	Job1	Job2	Job3	Job4	Job5	Job6	Job7	Job8	Job9	Job10
Job1 		-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	1.0 	1.0 	0.0 	
Job2 	-0.0 		-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	1.0 	
Job3 	-0.0 	-0.0 		-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	
Job4 	-